# Avocado Test
This script is supposed to be make sure that the created parquet file contains in fact the same information as in the xml/zip files of the avocado dataset

In [69]:
import pandas as pd
import zipfile
import re
import os

In [16]:
df = pd.read_parquet('avocado-1.0.2/avocado_light.parquet', engine='pyarrow')

In [49]:
df.head()

,messageid,sender_access,outlook_sender_name,outlook_recipient_name,sentto_address,cc_address,subject,processed_subject,sent_date,arrival_date,body_file_path,body_content,in_reply_to,Keywords
0,<FC38A9406AA4D411AB62009027DE9DA5E6545D@HQEXCH01>,EX,Fortunata Hermoso,Meshele Ko,Meshele Ko,'ray_rahamin@hp.com'; Dan Baca; Elba Linscott,Fw: RE: AvocadoIT's CEO committed to winning B...,AvocadoIT's CEO committed to winning Banamex deal,2001-04-12T22:02:41Z,2001-04-12T22:02:41Z,text/162/162-000001-EM.txt,"From: ""Fortunata Hermoso"" <fortunata.hermoso@a...",None,None
1,<19B3B310D020D311B57E00105A9A55241D5975@COFFEE>,EX,Paiman Komeilizadeh,Meshele Ko,All Employees,None,Expense reimbursement request- Amendment,Expense reimbursement request- Amendment,2000-04-24T23:06:06Z,2000-04-24T23:06:15Z,text/162/162-000003-EM.txt,"From: ""Paiman Komeilizadeh"" <pkomeilizadeh@avo...",None,None
2,<F57EA917353BD411B5BA00D0B708160C121B24@COFFEE>,EX,Meshele Ko,None,Piyush Goel; Dan Baca,None,FW: Information,Information,2000-06-20T20:51:04Z,2000-06-20T20:51:04Z,text/162/162-000004-EM.txt,"From: ""Meshele Ko"" <mko@avocadoit.com>\r\nTo: ...",None,None
3,<9640A23C9075D411B5CB00D0B708160C23A93D@COFFEE>,EX,Meshele Ko,None,'jbmcbryde@fedex.com',None,Fedex,Fedex,2000-09-12T17:01:08Z,2000-09-12T17:01:08Z,text/162/162-000005-EM.txt,"From: ""Meshele Ko"" <mko@avocadoit.com>\r\nTo: ...",None,None
4,<019D8990549CD3118A30009027D609CD25BF69@EXSRVR>,SMTP,Cindy Hilker,Meshele Ko,Meshele Ko (E-mail),Wayne Thayer,Icon and graphic for our WAP APPs,Icon and graphic for our WAP APPs,2000-05-03T01:27:13Z,2000-05-03T01:30:54Z,text/162/162-000006-EM.txt,Received: from exsrvr.homebid.com (197-175-161...,None,None


In [65]:
filtered_df = df[pd.isna(df['body_file_path'])]

In [67]:
filtered_df.head()

,messageid,sender_access,outlook_sender_name,outlook_recipient_name,sentto_address,cc_address,subject,processed_subject,sent_date,arrival_date,body_file_path,body_content,in_reply_to,Keywords
91,None,None,None,None,None,None,None,None,None,None,None,None,None,None
313,None,None,None,None,None,None,None,None,None,None,None,None,None,None
11985,None,None,None,None,None,None,None,None,None,None,None,None,None,None
40542,None,None,None,None,None,None,None,None,None,None,None,None,None,None
40858,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [63]:
ctr = 0
def test_function(row):
    global ctr

    file_path = row['body_file_path']
    print(file_path)
    # Split the filepath by slashes
    parts = file_path.split('/')

    # Extract the first two non-empty parts
    first_two_parts = [part for part in parts if part][:2]
    txt_file = [part for part in parts if part][1:3]
    # Join the first two parts with slashes
    zip_path = '/'.join(first_two_parts)+".zip"
    txt_path = '/'.join(txt_file)
    path_to_data = 'avocado-1.0.2/data/'


    with zipfile.ZipFile(path_to_data+zip_path, 'r') as zip_ref:
        with zip_ref.open(txt_path) as file:
            content = file.read()
            text = content.decode('utf-8')
            if repr(text) != repr(row['body_content']):
                ctr += 1
                print('not same')

    return None

In [64]:
df.apply(test_function, axis=1)

text/162/162-000001-EM.txt
text/162/162-000003-EM.txt
text/162/162-000004-EM.txt
text/162/162-000005-EM.txt
text/162/162-000006-EM.txt
text/162/162-000008-EM.txt
text/162/162-000009-EM.txt
text/162/162-000010-EM.txt
text/162/162-000013-EM.txt
text/162/162-000015-EM.txt
text/162/162-000017-EM.txt
text/162/162-000018-EM.txt
text/162/162-000019-EM.txt
text/162/162-000020-EM.txt
text/162/162-000023-EM.txt
text/162/162-000024-EM.txt
text/162/162-000025-EM.txt
text/162/162-000027-EM.txt
text/162/162-000029-EM.txt
text/162/162-000030-EM.txt
text/162/162-000032-EM.txt
text/162/162-000034-EM.txt
text/162/162-000035-EM.txt
text/162/162-000037-EM.txt
text/162/162-000038-EM.txt
text/162/162-000039-EM.txt
text/162/162-000040-EM.txt
text/162/162-000043-EM.txt
text/162/162-000045-EM.txt
text/162/162-000046-EM.txt
text/162/162-000048-EM.txt
text/162/162-000050-EM.txt
text/162/162-000051-EM.txt
text/162/162-000052-EM.txt
text/162/162-000053-EM.txt
text/162/162-000055-EM.txt
text/162/162-000056-EM.txt
t

AttributeError: 'NoneType' object has no attribute 'split'

In [71]:


def get_txt_files_from_zip(zip_path):
    """Extracts and returns a list of .txt filenames from a given ZIP file."""
    txt_files = []
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Iterate over all files in the ZIP archive
        for file_info in zip_ref.infolist():
            # Check if the file ends with .txt
            if file_info.filename.endswith('.txt'):
                txt_files.append(file_info.filename)
    return txt_files

def get_all_txt_files_from_directory(directory_path):
    """Compiles a list of all .txt filenames within ZIP files in a given directory."""
    all_txt_files = []
    # List all files in the directory
    for filename in os.listdir(directory_path):
        # Check if the file is a ZIP file
        if filename.endswith('.zip'):
            zip_path = os.path.join(directory_path, filename)
            # Get .txt files from this ZIP file
            txt_files = get_txt_files_from_zip(zip_path)
            all_txt_files.extend(txt_files)
    return all_txt_files

# Directory containing ZIP files
directory_path = 'avocado-1.0.2/data/text'

# Get all .txt filenames from ZIP files in the directory
txt_file_list = get_all_txt_files_from_directory(directory_path)



In [81]:
test = txt_file_list[:2500]

In [79]:
test

['185/185-001883-AP.txt',
 '185/185-001565-AP.txt',
 '185/185-000867-AP.txt',
 '185/185-001036-AP.txt',
 '185/185-000286-AP.txt',
 '185/185-000700-AP.txt',
 '185/185-001005-AP.txt',
 '185/185-001320-AP.txt',
 '185/185-001099_2-AT.txt',
 '185/185-001650-AP.txt',
 '185/185-001368-AP.txt',
 '185/185-000704-AP.txt',
 '185/185-001231-EM.txt',
 '185/185-000060-EM.txt',
 '185/185-001992-AP.txt',
 '185/185-001150_1-AT.txt',
 '185/185-001877-AP.txt',
 '185/185-000800-AP.txt',
 '185/185-001903-AP.txt',
 '185/185-000420-AP.txt',
 '185/185-000348-AP.txt',
 '185/185-000068-EM.txt',
 '185/185-001964-EM.txt',
 '185/185-000842-AP.txt',
 '185/185-000916-AP.txt']

In [85]:
ctr_true = 0
ctr_false = 0

not_included = []

for value in test:
    value = "text/"+value
    contains_value = value in df['body_file_path'].values
    if contains_value: 
        ctr_true += 1
        
    else: 
        not_included.append(value)
        ctr_false += 1
print("True: "+str(ctr_true)+"; False: "+str(ctr_false))

True: 770; False: 1730


In [86]:
not_included

['text/185/185-001883-AP.txt',
 'text/185/185-001565-AP.txt',
 'text/185/185-000867-AP.txt',
 'text/185/185-001036-AP.txt',
 'text/185/185-000286-AP.txt',
 'text/185/185-000700-AP.txt',
 'text/185/185-001005-AP.txt',
 'text/185/185-001320-AP.txt',
 'text/185/185-001099_2-AT.txt',
 'text/185/185-001650-AP.txt',
 'text/185/185-001368-AP.txt',
 'text/185/185-000704-AP.txt',
 'text/185/185-001992-AP.txt',
 'text/185/185-001150_1-AT.txt',
 'text/185/185-001877-AP.txt',
 'text/185/185-000800-AP.txt',
 'text/185/185-001903-AP.txt',
 'text/185/185-000420-AP.txt',
 'text/185/185-000348-AP.txt',
 'text/185/185-000842-AP.txt',
 'text/185/185-000916-AP.txt',
 'text/185/185-001999-AP.txt',
 'text/185/185-001779-AP.txt',
 'text/185/185-001796-AP.txt',
 'text/185/185-001293-AP.txt',
 'text/185/185-001262-AP.txt',
 'text/185/185-001297-AP.txt',
 'text/185/185-000371-AP.txt',
 'text/185/185-001513-AP.txt',
 'text/185/185-001658-AP.txt',
 'text/185/185-000324-AP.txt',
 'text/185/185-001587-AP.txt',
 'te

In [76]:
df.head()

,messageid,sender_access,outlook_sender_name,outlook_recipient_name,sentto_address,cc_address,subject,processed_subject,sent_date,arrival_date,body_file_path,body_content,in_reply_to,Keywords
0,<FC38A9406AA4D411AB62009027DE9DA5E6545D@HQEXCH01>,EX,Fortunata Hermoso,Meshele Ko,Meshele Ko,'ray_rahamin@hp.com'; Dan Baca; Elba Linscott,Fw: RE: AvocadoIT's CEO committed to winning B...,AvocadoIT's CEO committed to winning Banamex deal,2001-04-12T22:02:41Z,2001-04-12T22:02:41Z,text/162/162-000001-EM.txt,"From: ""Fortunata Hermoso"" <fortunata.hermoso@a...",None,None
1,<19B3B310D020D311B57E00105A9A55241D5975@COFFEE>,EX,Paiman Komeilizadeh,Meshele Ko,All Employees,None,Expense reimbursement request- Amendment,Expense reimbursement request- Amendment,2000-04-24T23:06:06Z,2000-04-24T23:06:15Z,text/162/162-000003-EM.txt,"From: ""Paiman Komeilizadeh"" <pkomeilizadeh@avo...",None,None
2,<F57EA917353BD411B5BA00D0B708160C121B24@COFFEE>,EX,Meshele Ko,None,Piyush Goel; Dan Baca,None,FW: Information,Information,2000-06-20T20:51:04Z,2000-06-20T20:51:04Z,text/162/162-000004-EM.txt,"From: ""Meshele Ko"" <mko@avocadoit.com>\r\nTo: ...",None,None
3,<9640A23C9075D411B5CB00D0B708160C23A93D@COFFEE>,EX,Meshele Ko,None,'jbmcbryde@fedex.com',None,Fedex,Fedex,2000-09-12T17:01:08Z,2000-09-12T17:01:08Z,text/162/162-000005-EM.txt,"From: ""Meshele Ko"" <mko@avocadoit.com>\r\nTo: ...",None,None
4,<019D8990549CD3118A30009027D609CD25BF69@EXSRVR>,SMTP,Cindy Hilker,Meshele Ko,Meshele Ko (E-mail),Wayne Thayer,Icon and graphic for our WAP APPs,Icon and graphic for our WAP APPs,2000-05-03T01:27:13Z,2000-05-03T01:30:54Z,text/162/162-000006-EM.txt,Received: from exsrvr.homebid.com (197-175-161...,None,None


In [89]:
id = "<11EC3A0EF482D311ABD40008C7E64C700CDF2D05@diana.portal.com>"

df[df['messageid'] == id]

,messageid,sender_access,outlook_sender_name,outlook_recipient_name,sentto_address,cc_address,subject,processed_subject,sent_date,arrival_date,body_file_path,body_content,in_reply_to,Keywords
910188,<11EC3A0EF482D311ABD40008C7E64C700CDF2D05@dian...,SMTP,Phil Wolf,Peter Smialek,Peter Smialek (E-mail),None,VP Presentation,VP Presentation,2002-03-04T21:30:00Z,2002-03-04T21:39:18Z,text/185/185-000001-EM.txt,Received: from mailer.avocadoit.com (10.7.1.4 ...,None,None
